In [5]:
from nupack import *
import random
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp as mt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
bases = ["A","C","G","T"]
bigcounter = 0
successes = 0

my_model = Model(material='dna', celsius=25, sodium=0.05, magnesium=0.01)

while True:
    found = False

    while not found:
        
        bigcounter += 1
        if bigcounter%1000 == 0:
            print(bigcounter)

        fine = True
        
        #Generates the random array used to build imagers and handles according to the scheme
        pos= [0]*15

        pos[0] = random.sample(bases,1)

        for i in range(1,13):
            pos[i] = random.sample(bases,3)

        pos[13] = random.sample(bases,2)
        pos[14] = random.sample(bases,1)
        
        # This is the scheme
        ImA = [pos[0][0],pos[1][0],pos[2][0],pos[3][0],pos[4][0],pos[5][1],pos[6][1],pos[7][1],pos[8][1],pos[9][1],pos[10][1],pos[11][1],pos[12][1],pos[13][0],pos[14][0]]
        ImB = [pos[0][0],pos[1][1],pos[2][1],pos[3][1],pos[4][1],pos[5][0],pos[6][0],pos[7][0],pos[8][0],pos[9][1],pos[10][1],pos[11][1],pos[12][1],pos[13][0],pos[14][0]]
        ImC = [pos[0][0],pos[1][1],pos[2][1],pos[3][1],pos[4][1],pos[5][1],pos[6][1],pos[7][1],pos[8][1],pos[9][0],pos[10][0],pos[11][0],pos[12][0],pos[13][0],pos[14][0]]

        imagers = [[ImA, 'a'], [ImB, 'b'], [ImC,'c']]

        H_A = [pos[0][0],pos[1][0],pos[2][0],pos[3][0],pos[4][0],pos[5][2],pos[6][1],pos[7][1],pos[8][1],pos[9][1],pos[10][1],pos[11][2],pos[12][2],pos[13][1],pos[14][0]]
        H_B = [pos[0][0],pos[1][2],pos[2][2],pos[3][1],pos[4][1],pos[5][0],pos[6][0],pos[7][0],pos[8][0],pos[9][1],pos[10][1],pos[11][1],pos[12][2],pos[13][1],pos[14][0]]
        H_C = [pos[0][0],pos[1][2],pos[2][1],pos[3][1],pos[4][1],pos[5][1],pos[6][1],pos[7][2],pos[8][2],pos[9][0],pos[10][0],pos[11][0],pos[12][0],pos[13][1],pos[14][0]]

        H_AB = [pos[0][0],pos[1][0],pos[2][0],pos[3][0],pos[4][0],pos[5][0],pos[6][0],pos[7][0],pos[8][0],pos[9][1],pos[10][1],pos[11][1],pos[12][1],pos[13][0],pos[14][0]]
        H_BC = [pos[0][0],pos[1][1],pos[2][1],pos[3][1],pos[4][1],pos[5][0],pos[6][0],pos[7][0],pos[8][0],pos[9][0],pos[10][0],pos[11][0],pos[12][0],pos[13][0],pos[14][0]]
        H_AC = [pos[0][0],pos[1][0],pos[2][0],pos[3][0],pos[4][0],pos[5][1],pos[6][1],pos[7][1],pos[8][1],pos[9][0],pos[10][0],pos[11][0],pos[12][0],pos[13][0],pos[14][0]]

        H_ABC = [pos[0][0],pos[1][1],pos[2][1],pos[3][1],pos[4][1],pos[5][1],pos[6][1],pos[7][1],pos[8][1],pos[9][1],pos[10][1],pos[11][1],pos[12][1],pos[13][0],pos[14][0]]

        handles = [[H_A,'a'], [H_B,'b'], [H_C,'c'], [H_AB,'a','b'], [H_BC,'b','c'], [H_AC,'a','c'], [H_ABC,'a','b','c']]
        
        # I randomly shuffle the (inner) positions, so the mismatches are not always in the same position.
        
        shuffle = [1,2,3,4,5,6,7,8,9,10,11,12,13]
        random.shuffle(shuffle)
        shuffle = [0] + shuffle + [14]

        # Now I generate the shuffled strands by picking, for each position, the base in the unshuffled strand that is pointed to by the shuffle list

        for im in range(3):
            new_im = [0]*15
            for pos in range(15):
                new_im[pos] = imagers[im][0][shuffle[pos]]
            imagers[im][0] = new_im

        for han in range(7):
            new_handle = [0]*15
            for pos in range(15):
                new_handle[pos] = handles[han][0][shuffle[pos]]
            handles[han][0] = new_handle

        for im in range(3):
            imagers[im][0] = Seq(''.join(imagers[im][0]))

        for han in range(7):
            handles[han][0] = Seq(''.join(handles[han][0]))

        all_matches = [[0]*6, [0]*6, [0]*6]
        
        # I now check for matches by running pairwise alignments with parameters that count matches as one, mismatches as 0, and gaps opening and lengthening both as -1.
        # I use this as a surrogate of how well the two sequences should bind. The threshold can be adjusted, but 5 or 6 are working numbers
        
        for i in range(3):
            for j in range(6):
                align = pairwise2.align.localms(imagers[i][0], handles[j][0], 1,-0,-1,-1,score_only=True)
                all_matches[i][j] = align
                if (align > 5 and imagers[i][1] not in handles[j]):
                    fine = False

        if not fine:
            # print('align')
            continue
        
        # I now check for the melting temperature of the pairs of wnated bindings. I want them to be in a certain range and if too many of them are not in this range
        # I reject the set. The algorithm does not accept end mismatches, which is why I don't allow for them when shuffling. It also lacks parameters for most double
        # mismatches, which is why I check if there are any for a certain binding and I don't check it in that case. This could mean I end up not actually checking any
        # sequence, which is why I also require to have checked a certain number of them. Finally, there is another algorithm in biopython that approximates energies
        # even for end/double mismatches, but it's imprecise so I don't know if it makes sense to use it (it varied a lot when I tried it). The use of one or the other
        # can be toggled with the Strict variable.
        
        Strict = True
        tolerance = 5
        checked_tms = 0
        outliers = 0
        c_Na = 50
        c_Mg = 10
        c_Tris = 5

        if Strict:
            for i in range(3):
                for j in range(7):
                    if imagers[i][1] in handles[j]:
                        check = 1
                        bad = False
                        for k in range(15):
                            if imagers[i][0][k] == handles[j][0][k]:
                                check = 1
                            else:
                                if check == 0:
                                    bad = True
                                    break
                                else:
                                    check = 0
                        if bad: True
                        else:
                            tm = mt.Tm_NN(imagers[i][0], c_seq=handles[j][0][::-1].reverse_complement())
                            if tm < -45 or tm > -25:
                                if outliers < tolerance: outliers += 1
                                else: fine = False
                            else: checked_tms += 1
        else:
            for i in range(3):
                for j in range(7):
                    if imagers[i][1] in handles[j]:
                        tm = mt.Tm_NN(imagers[i][0], c_seq=handles[j][0][::-1].reverse_complement(), strict=False)
                        if tm < -45 or tm > -15:
                            if outliers < tolerance: outliers += 1
                            else: fine = False

        if not fine or checked_tms<3: #add " or checked_tms<3" oly if using the Strict version
            # print('tm')
            continue
        
        # Here I do the same check I did for unwanted bindings, but for pairs of imaging strands (them binding would influence the effective amount of strand in solution)
        
        dimer_matches = [[0]*3, [0]*3, [0]*3]

        for i in range(3):
            for j in range(3):
                dimer_matches[i][j] = pairwise2.align.localms(imagers[i][0], imagers[j][0].reverse_complement(), 1,-0,-1,-1, score_only = True)
                if dimer_matches[i][j] > 6: fine = False

        if not fine:
            # print('dimer')
            continue
            
        # Check if an imaging strand forms a stem by trying to align the first 6 bases and the reverse complement of the last 6 for each imaging strand

        stem_matches = [0,0,0]

        for i in range(3):
            stem_matches[i] = pairwise2.align.localms(Seq(str(imagers[i][0])[:6]), Seq(str(imagers[i][-6:])).reverse_complement(), 1,-0,-1,-1, score_only = True)
            if stem_matches[i] > 3: fine = False

        if not fine:
            # print('stem')
            continue

        found = True
    
    # If a set passes all these tests I consider it a success, pass it to NUPACK and calculate the tube ensemble for it, save the tube results, and plot the energy matruces
    successes += 1
    print("SUCCESS:" + str(successes))
    
    imagers_clean = []
    for im in imagers:
        imagers_clean.append(str(im[0]))
        
    rev_comp_handles = []
    for han in handles:
        rev_comp_handles.append(str(han[0].reverse_complement()))
    
    ImA = Strand(imagers_clean[0], name='ImA')
    ImB = Strand(imagers_clean[1], name='ImB')
    ImC = Strand(imagers_clean[2], name='ImC')
    
    HA = Strand(rev_comp_handles[0], name='HA')
    HB = Strand(rev_comp_handles[1], name='HB')
    HC = Strand(rev_comp_handles[2], name='HC')
    HAB = Strand(rev_comp_handles[3], name='HAB')
    HBC = Strand(rev_comp_handles[4], name='HBC')
    HAC = Strand(rev_comp_handles[5], name='HAC')
    HABC = Strand(rev_comp_handles[6], name='HABC')
    
    t1 = Tube(strands={ImA:1e-6, ImB:1e-6, ImC:1e-6, HA:1e-6, HB:1e-6, HC:1e-6, HAB:1e-6, HBC:1e-6, HAC:1e-6, HABC:1e-6}, complexes=SetSpec(max_size=2), name='Tube t1')
    
    tube_result = tube_analysis(tubes=[t1], compute=['pairs'], model=my_model)
    
    tube_result.save_text('my_result'+str(successes)+'.txt')
    
    correctness_matrix = np.array([1,0,0],[0,1,0],[0,0,1],[1,1,0],[0,1,1],[1,0,1],[1,1,1])
    correctness_df = pd.DataFrame((correctness_matrix), columns=imagers, index=handles)
    energy_matrix = np.empty((7,3))
    energy_df = pd.DataFrame(energy_matrix, columns=imagers, index=handles)

    for imager in imagers:
        for handle in handles:
            energy_df.loc[handle][imager] = df.loc[df[imager] & df[handle] == 1]["delta_G"]

    fig, (ax1, ax2) = plt.subplots(1,2)
    im1 = ax1.imshow(correctness_df, cmap='hot')
    im2 = ax2.imshow(energy_df, cmap='Greys')
    fig.subplots_adjust(right=0.6)
    cbar_ax = fig.add_axes([0.65, 0.15, 0.05, 0.7])
    fig.colorbar(im2, cax=cbar_ax)
    for _ in range(len(imagers)):
        print(imagernames[_]+': ',im_strings[_], pfunc(strands=[imagers[_]], model=my_model)[1])
    for _ in range(len(handles)):
        print(handlenames[_]+': ',han_strings[_], pfunc(strands=[han_strings[_]], model=my_model)[1])
    print(energy_df)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


KeyboardInterrupt: 